In [1]:
import pandas as pd
import numpy as np

/usr/local/lib/python3.5/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [4]:
idt = pd.read_json('https://raw.githubusercontent.com/Gapminder/waffle-server-importers-exporters/world-legacy-with-data/data/out/gw/meta/quantities.json?token=AAVecNfSWC2bIGcLqoUZKjOVZIPQpm6Kks5W66t2wA%3D%3D')

In [179]:
import re


def to_concept_id(s, sub='[/ -\.\*";]+', sep='_'):
    '''convert a string to lowercase alphanumeric + underscore id for concepts'''
    s1 = re.sub(sub, sep, s.strip())
#     s1 = re.sub(r'\[.*\]', '', s1)
    s1 = s1.replace('\n', '')

    if s1[-1] == sep:  # remove the last underscore
        s1 = s1[:-1]

    return s1.lower()


In [7]:
idt.to_csv('ddf--concepts.csv', index=False)

In [22]:
concepts = pd.read_csv('./source/graph_settings - Indicators.csv')

In [23]:
dsc = concepts.columns

In [24]:
concepts.columns = list(map(to_concept_id, concepts.columns))

In [25]:
concepts['concept_type'] = 'measure'

In [26]:
concepts = concepts.drop('download', axis=1)

In [27]:
concepts = concepts.iloc[:, [5, 0, 1,2,3,4,6,7]]

In [30]:
concepts = concepts.rename(columns={'ddf_id':'concept'})

In [31]:
concepts.head()

,concept_id,name,tooltip,menu_level1,menu_level_2,indicator_url,scale,concept_type
0,children_per_woman_total_fertility,Children per woman (total fertility),Total fertility rate. The number of children t...,NaN,NaN,http://spreadsheets.google.com/pub?key=phAwcNA...,lin,measure
1,co2_emissions_tonnes_per_person,CO2 emissions (tonnes per person),Carbon dioxide emissions from the burning of f...,NaN,NaN,http://spreadsheets.google.com/pub?key=phAwcNA...,lin,measure
2,income_per_person_gdppercapita_ppp_inflation_a...,"Income per person (GDP/capita, PPP$ inflation-...",Gross domestic product per person adjusted for...,NaN,NaN,http://spreadsheets.google.com/pub?key=phAwcNA...,log,measure
3,child_mortality_0_5_year_olds_dying_per_1000_born,"Child mortality (0-5 year-olds dying per 1,000...",Death of children under five years of age per ...,NaN,NaN,http://spreadsheets.google.com/pub?key=0ArfEDs...,log,measure
4,life_expectancy_years,Life expectancy (years),The average number of years a newborn child wo...,NaN,NaN,http://spreadsheets.google.com/pub?key=phAwcNA...,lin,measure


In [34]:
dsc.drop('ddf_id')

Index(['Name', 'Tooltip', 'Menu level1', 'Menu level 2', 'Indicator url',
       'Download', 'Scale'],
      dtype='object')

In [35]:
len(concepts)

544

In [255]:
cc = concepts[['concept_id', 'name', 'concept_type']].copy()

In [256]:
cc.columns = [['concept', 'name', 'concept_type']]

In [257]:
k = cc[cc.concept == u'———————————————————————'].index

In [258]:
cc = cc.drop(k)

In [261]:
cc['drill_up'] = np.nan
cc['domain'] = np.nan

In [300]:
cc.to_csv('./output/ddf--concepts--continuous.csv', index=False)

In [259]:
# todo: discrete concepts

In [283]:
dc = pd.DataFrame([], columns=cc.columns)

In [278]:
dcl = list(map(to_concept_id, gps.n.values))

In [284]:
dcl_ = np.r_[dcl, ['geo', 'country','time', 'name'], [x+'_id' for x in dcl]]
dcl_2 = np.r_[gps.n.values, ['Geo', 'Country','Time', 'Name'], [x+' id' for x in gps.n.values]]

In [269]:
dcl_

array(['geographic_regions', 'income_groups', 'landlocked',
       'g77_and_oecd_countries', 'geographic_regions_in_4_colors',
       'main_religion_2008', 'time', 'name', 'geographic_regions_id',
       'income_groups_id', 'landlocked_id', 'g77_and_oecd_countries_id',
       'geographic_regions_in_4_colors_id', 'main_religion_2008_id'], 
      dtype='<U33')

In [285]:
dc['concept'] = dcl_
dc['name'] = dcl_2

In [295]:
dc['concept_type'] = 'string'
dc.loc[:5, 'concept_type'] = 'entity set'
dc.loc[6, 'concept_type'] = 'entity domain'
dc.loc[7, 'concept_type'] = 'entity set'
dc.loc[8, 'concept_type'] = 'time'

In [302]:
# dc['concept_type'] = 'string'
dc.loc[:5, 'domain'] = 'geo'

dc.loc[7, 'drill_up'] = dcl
dc.loc[7, 'domain'] = 'geo'

In [303]:
dc

,concept,name,concept_type,drill_up,domain
0,geographic_regions,Geographic regions,entity set,NaN,geo
1,income_groups,Income groups,entity set,NaN,geo
2,landlocked,Landlocked,entity set,NaN,geo
3,g77_and_oecd_countries,G77 and OECD countries,entity set,NaN,geo
4,geographic_regions_in_4_colors,Geographic regions in 4 colors,entity set,NaN,geo
5,main_religion_2008,Main religion 2008,entity set,NaN,geo
6,geo,Geo,entity domain,NaN,NaN
7,country,Country,entity set,"[geographic_regions, income_groups, landlocked...",geo
8,time,time,time,NaN,NaN
9,name,name,string,NaN,NaN


In [304]:
dc.to_csv('./output/ddf--concepts--discrete.csv', index=False)

In [55]:
gps = pd.read_json('../../ddf--gapminder_world/waffle-server-importers-exporters-world-legacy-with-data/data/out/gw/meta/area_categorizarion.json')

In [58]:
gps

,a,dataCollectionUrl,groupings,id,n,providerUrl,sourceName
0,Regions of the world,http://spreadsheets.google.com/pub?key=phT4mwj...,"{'i265': ['i5', 'i13', 'i31', 'i35', 'i44', 'i...",CATID0,Geographic regions,http://spreadsheets.google.com/pub?key=phT4mwj...,http://spreadsheets.google.com/pub?key=phT4mwj...
1,How rich are you?,http://spreadsheets.google.com/pub?key=phT4mwj...,"{'i268': ['i6', 'i9', 'i12', 'i13', 'i14', 'i1...",CATID1,Income groups,http://spreadsheets.google.com/pub?key=phT4mwj...,http://spreadsheets.google.com/pub?key=phT4mwj...
2,Does the country have a coast?,http://spreadsheets.google.com/pub?key=rBwKBod...,"{'i270': ['i2', 'i3', 'i4', 'i5', 'i7', 'i8', ...",CATID2,Landlocked,http://spreadsheets.google.com/pub?key=rBwKBod...,http://spreadsheets.google.com/pub?key=rBwKBod...
3,Member of the G77 (developing country) group o...,http://spreadsheets.google.com/pub?key=tVI_UCw...,"{'i273': ['i13', 'i14', 'i21', 'i37', 'i57', '...",CATID3,G77 and OECD countries,http://spreadsheets.google.com/pub?key=tVI_UCw...,http://spreadsheets.google.com/pub?key=tVI_UCw...
4,Using 4 colors to describe the geographic regi...,http://spreadsheets.google.com/spreadsheet/pub...,"{'i277': ['i4', 'i7', 'i23', 'i28', 'i33', 'i3...",CATID4,Geographic regions in 4 colors,http://spreadsheets.google.com/spreadsheet/pub...,http://spreadsheets.google.com/spreadsheet/pub...
5,The religion of more than 50% of the religious...,http://spreadsheets.google.com/spreadsheet/pub...,"{'i279': ['i5', 'i6', 'i7', 'i8', 'i9', 'i10',...",CATID5,Main religion 2008,http://spreadsheets.google.com/spreadsheet/pub...,http://spreadsheets.google.com/spreadsheet/pub...


In [57]:
gps['groupings'][1]

{'i266': ['i1',
  'i18',
  'i23',
  'i33',
  'i34',
  'i35',
  'i40',
  'i41',
  'i48',
  'i49',
  'i68',
  'i71',
  'i80',
  'i92',
  'i93',
  'i95',
  'i114',
  'i116',
  'i120',
  'i125',
  'i132',
  'i133',
  'i136',
  'i151',
  'i152',
  'i155',
  'i162',
  'i185',
  'i202',
  'i207',
  'i222',
  'i223',
  'i226',
  'i236',
  'i257',
  'i211'],
 'i267': ['i3',
  'i4',
  'i5',
  'i7',
  'i10',
  'i15',
  'i20',
  'i22',
  'i27',
  'i28',
  'i29',
  'i32',
  'i44',
  'i47',
  'i52',
  'i55',
  'i61',
  'i62',
  'i64',
  'i74',
  'i79',
  'i87',
  'i99',
  'i103',
  'i104',
  'i109',
  'i112',
  'i113',
  'i123',
  'i126',
  'i131',
  'i134',
  'i135',
  'i138',
  'i141',
  'i143',
  'i148',
  'i153',
  'i171',
  'i172',
  'i175',
  'i183',
  'i189',
  'i191',
  'i198',
  'i201',
  'i209',
  'i215',
  'i224',
  'i228',
  'i231',
  'i232',
  'i233',
  'i235',
  'i246'],
 'i268': ['i6',
  'i9',
  'i12',
  'i13',
  'i14',
  'i16',
  'i17',
  'i19',
  'i21',
  'i24',
  'i31',
  'i37',
  

In [60]:
geo = pd.read_excel('../waffle-server-importers-exporters-world-legacy-with-data/data/synonym/country_synonyms.xlsx')

In [64]:
geo.head() 

,Use Name,Gapminder list,Alternative 1,Alternative 2,Alternative 3,Alternative 4 (CDIAC),pandg,GOD_ID,ISO3dig_ext,ALT 5,NAME,CODE,Number,Gwid,arb1,arb2,arb3,arb4,arb5,arb6
0,Abkhazia,Abkhazia,NaN,NaN,NaN,NaN,NaN,GE-AB,_abk,NaN,NaN,NaN,NaN,i0,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,Afghanistan,Islamic Republic of Afghanistan,NaN,NaN,Afghanistan,AFGHANISTAN,AF,afg,NaN,AFGHANISTAN,AFG,4,i1,NaN,NaN,NaN,NaN,NaN,NaN
2,Akrotiri and Dhekelia,Akrotiri and Dhekelia,NaN,NaN,NaN,NaN,NaN,Akrotiri_Dhekelia,_akr,NaN,NaN,NaN,NaN,i2,NaN,NaN,NaN,NaN,NaN,NaN
3,Albania,Albania,NaN,NaN,NaN,Albania,ALBANIA,AL,alb,NaN,ALBANIA,ALB,8,i3,NaN,NaN,NaN,NaN,NaN,NaN
4,Algeria,Algeria,NaN,NaN,NaN,Algeria,ALGERIA,DZ,dza,NaN,ALGERIA,DZA,12,i4,NaN,NaN,NaN,NaN,NaN,NaN


In [65]:
geo_ = geo[['ISO3dig_ext', 'Gwid']]

In [66]:
geo_ = geo_.set_index('Gwid')

In [70]:
geo_.get_value('i0', 'ISO3dig_ext')

'_abk'

In [110]:
def rename_col(s):
    
    real = idt[idt['-t-ind'] == s]['-t-name'].iloc[0]
    cid = concepts[concepts['name'] == real]['concept_id'].iloc[0]
#     print(real, ': ', cid)
    
    return cid

In [92]:
def rename_geo(s):
    return geo_.get_value(s, 'ISO3dig_ext')

In [71]:
import os

In [73]:
path = '../waffle-server-importers-exporters-world-legacy-with-data/data/out/gw/indicators/'

In [74]:
fs = os.listdir(path)

In [95]:
p1 = os.path.join(path, fs[300])
d1 = pd.read_json(p1)

In [96]:
d1.geo = d1.geo.apply(rename_geo)

In [97]:
d1

,geo,rCyfwvThkbHVlNVw48vHybg,time
0,afg,69.5,1991
1,afg,69.3,1992
2,afg,69.7,1993
3,afg,69.3,1994
4,afg,69.3,1995
5,afg,69.4,1996
6,afg,69.5,1997
7,afg,69.7,1998
8,afg,69.6,1999
9,afg,69.6,2000


In [85]:
idt[idt['-t-ind'] == '0AgogXXPMARyldC1rcTI5OU50Mnc1djdkNXpnWUFrZmc']['-t-name'].iloc[0]

'Murdered children 0-14 (per 100,000 people)'

In [115]:
rename_col('rCyfwvThkbHVlNVw48vHybg')

'males_aged_15_24_employment_rate_percent'

In [ ]:
# now try to make data points

In [114]:
for f in fs[1:]:
    p = os.path.join(path, f)
    
    col = f[:-5]
    col_r = rename_col(col)
  
    d = pd.read_json(p)

    if 'geo' in d.columns:
        d['geo'] = d['geo'].apply(rename_geo)
        d = d.rename(columns={col:col_r}) #ok?
        d.to_csv('./output/datapoints/ddf--datapoints--'+col_r+'--by--geo--time.csv', index=False)
    else:
        print(col, ':', col_r)
    
    

rj8WsGGNXkijP447qnY5RLg : ———————————————————————
rMF2H57_Vd8TdobvtNhYuwg : ———————————————————————
tAwlNo30o6vVgDUk0EmDolQ : ———————————————————————
tDkfgIt5oQ_ZJQ7IBmUvsUg : ———————————————————————
tdKPTXS1U5Jn3qAQICk4-Vw : ———————————————————————
teiT4rPfnWcd5mCldIm4Zhw : ———————————————————————
tFc2tyrcWLcUyGhq6mTAbDg : ———————————————————————
tFrUv6vnhhO4Jr1czdG_p0Q : ———————————————————————
tIL264Nlw0AlIWlXuCWWNmQ : ———————————————————————
tJwP0x54AZ-X-bGL-RebrpA : ———————————————————————
tLiJLI_rimqrdQRWdamPPRQ : ———————————————————————
tnDimTzGwU_BbmtOMHOUFFQ : ———————————————————————
ts7TQEIg0amXovCDzQxV5EA : ———————————————————————
tYvQLeBTuoKsyofvwGqS-9w : ———————————————————————


In [102]:
fs[1][:-5]

'0AgogXXPMARyldC1rcTI5OU50Mnc1djdkNXpnWUFrZmc'

In [106]:
len(idt)

542

In [107]:
len(concepts)

544

In [116]:
# entity

In [118]:
gps

,a,dataCollectionUrl,groupings,id,n,providerUrl,sourceName
0,Regions of the world,http://spreadsheets.google.com/pub?key=phT4mwj...,"{'i265': ['i5', 'i13', 'i31', 'i35', 'i44', 'i...",CATID0,Geographic regions,http://spreadsheets.google.com/pub?key=phT4mwj...,http://spreadsheets.google.com/pub?key=phT4mwj...
1,How rich are you?,http://spreadsheets.google.com/pub?key=phT4mwj...,"{'i268': ['i6', 'i9', 'i12', 'i13', 'i14', 'i1...",CATID1,Income groups,http://spreadsheets.google.com/pub?key=phT4mwj...,http://spreadsheets.google.com/pub?key=phT4mwj...
2,Does the country have a coast?,http://spreadsheets.google.com/pub?key=rBwKBod...,"{'i270': ['i2', 'i3', 'i4', 'i5', 'i7', 'i8', ...",CATID2,Landlocked,http://spreadsheets.google.com/pub?key=rBwKBod...,http://spreadsheets.google.com/pub?key=rBwKBod...
3,Member of the G77 (developing country) group o...,http://spreadsheets.google.com/pub?key=tVI_UCw...,"{'i273': ['i13', 'i14', 'i21', 'i37', 'i57', '...",CATID3,G77 and OECD countries,http://spreadsheets.google.com/pub?key=tVI_UCw...,http://spreadsheets.google.com/pub?key=tVI_UCw...
4,Using 4 colors to describe the geographic regi...,http://spreadsheets.google.com/spreadsheet/pub...,"{'i277': ['i4', 'i7', 'i23', 'i28', 'i33', 'i3...",CATID4,Geographic regions in 4 colors,http://spreadsheets.google.com/spreadsheet/pub...,http://spreadsheets.google.com/spreadsheet/pub...
5,The religion of more than 50% of the religious...,http://spreadsheets.google.com/spreadsheet/pub...,"{'i279': ['i5', 'i6', 'i7', 'i8', 'i9', 'i10',...",CATID5,Main religion 2008,http://spreadsheets.google.com/spreadsheet/pub...,http://spreadsheets.google.com/spreadsheet/pub...


In [119]:
regs = pd.read_json('../waffle-server-importers-exporters-world-legacy-with-data/data/synonym/regions.json')

In [121]:
import json

In [123]:
reg_f = open('../waffle-server-importers-exporters-world-legacy-with-data/data/synonym/regions.json')

regs = json.load(reg_f)

In [129]:
regs

[{'Sub-Saharan Africa': 'i260'},
 {'South Asia': 'i261'},
 {'Middle East & North Africa': 'i262'},
 {'America': 'i263'},
 {'Europe & Central Asia': 'i264'},
 {'East Asia & Pacific': 'i265'},
 {'Low income': 'i266'},
 {'Upper middle income': 'i267'},
 {'High income': 'i268'},
 {'Lower middle income': 'i269'},
 {'Coastline': 'i270'},
 {'Landlocked': 'i271'},
 {'G77': 'i272'},
 {'OECD': 'i273'},
 {'Others': 'i274'},
 {'[Europe]': 'i275'},
 {'[Asia]': 'i276'},
 {'[Africa]': 'i277'},
 {'[America]': 'i278'},
 {'Christian': 'i279'},
 {'Muslim': 'i280'},
 {'Eastern religions': 'i281'}]

In [150]:
ttt = regs[0]

ttt.get(list(ttt.keys())[0])

'i260'

In [151]:
regd = {}

for i in regs:
    regd[i.get(list(i.keys())[0])] = list(i.keys())[0]

In [152]:
regd

{'i260': 'Sub-Saharan Africa',
 'i261': 'South Asia',
 'i262': 'Middle East & North Africa',
 'i263': 'America',
 'i264': 'Europe & Central Asia',
 'i265': 'East Asia & Pacific',
 'i266': 'Low income',
 'i267': 'Upper middle income',
 'i268': 'High income',
 'i269': 'Lower middle income',
 'i270': 'Coastline',
 'i271': 'Landlocked',
 'i272': 'G77',
 'i273': 'OECD',
 'i274': 'Others',
 'i275': '[Europe]',
 'i276': '[Asia]',
 'i277': '[Africa]',
 'i278': '[America]',
 'i279': 'Christian',
 'i280': 'Muslim',
 'i281': 'Eastern religions'}

In [131]:
gps.iloc[0]['groupings'].keys()

dict_keys(['i265', 'i263', 'i261', 'i260', 'i264', 'i262'])

In [132]:
gps.iloc[1]['groupings'].keys()

dict_keys(['i268', 'i269', 'i267', 'i266'])

In [133]:
gps.iloc[2]['groupings'].keys()

dict_keys(['i270', 'i271'])

In [134]:
gps.iloc[3]['groupings'].keys()

dict_keys(['i273', 'i272', 'i274'])

In [135]:
gps.iloc[4]['groupings'].keys()

dict_keys(['i277', 'i275', 'i278', 'i276'])

In [136]:
gps.iloc[5]['groupings'].keys()

dict_keys(['i279', 'i281', 'i280'])

In [153]:
gps.n.apply(to_concept_id)

0                geographic_regions
1                     income_groups
2                        landlocked
3            g77_and_oecd_countries
4    geographic_regions_in_4_colors
5                main_religion_2008
Name: n, dtype: object

In [137]:
country = geo_.copy()

In [180]:
country['geographic_regions'] = np.nan
country['income_groups'] = np.nan
country['landlocked'] = np.nan
country['g77_and_oecd_countries'] = np.nan
country['geographic_regions_in_4_colors'] = np.nan
country['main_religion_2008'] = np.nan

In [197]:
res = {}

for k, v in gps.iloc[4]['groupings'].items():
#     print(regd[k])
    for c in v:
        if c:
            res[c] = to_concept_id(regd[k], sub='[/ -\.\*";\[\]]+', sep='')
            
ser = pd.Series(res)

In [198]:
country['geographic_regions_in_4_colors'] = ser

In [200]:
country = country.reset_index()

In [210]:
country['is--country'] = 'TRUE'
country = country.rename(columns={'ISO3dig_ext':'country', 'Gwid':'gwid'})

In [211]:
country.head()

,gwid,country,geographic_regions,income_groups,landlocked,g77_and_oecd_countries,geographic_regions_in_4_colors,main_religion_2008,is--country
0,i0,_abk,europe_central_asia,NaN,NaN,others,europe,NaN,TRUE
1,i1,afg,south_asia,low_income,landlocked,g77,asia,muslim,TRUE
2,i2,_akr,europe_central_asia,NaN,coastline,others,europe,NaN,TRUE
3,i3,alb,europe_central_asia,upper_middle_income,coastline,others,europe,muslim,TRUE
4,i4,dza,middle_east_north_africa,upper_middle_income,coastline,g77,africa,muslim,TRUE


In [212]:
country.to_csv('./output/ddf--entities--geo--country.csv', index=False)

In [223]:
geographic_regions = pd.DataFrame([], columns=['geographic_region', 'name', 'geographic_region_id', 'is--geographic_region'])

geographic_regions['geographic_region_id'] = gps.iloc[0]['groupings'].keys()
geographic_regions['geographic_region'] = geographic_regions['geographic_region_id'].apply(lambda x: to_concept_id(regd[x], sub='[/ -\.\*";\[\]]+') )
geographic_regions['name'] = geographic_regions['geographic_region_id'].apply(lambda x: regd[x] )

geographic_regions['is--geographic_region'] = 'TRUE'

In [224]:
geographic_regions

,geographic_region,name,geographic_region_id,is--geographic_region
0,east_asia_pacific,East Asia & Pacific,i265,TRUE
1,america,America,i263,TRUE
2,south_asia,South Asia,i261,TRUE
3,sub_saharan_africa,Sub-Saharan Africa,i260,TRUE
4,europe_central_asia,Europe & Central Asia,i264,TRUE
5,middle_east_north_africa,Middle East & North Africa,i262,TRUE


In [236]:
geographic_regions.to_csv('./output/ddf--entities--geo--geographic_region.csv', index=False)

In [225]:
income_group = pd.DataFrame([], columns=['income_group', 'name', 'income_group_id', 'is--income_group'])

income_group['income_group_id'] = gps.iloc[1]['groupings'].keys()
income_group['income_group'] = income_group['income_group_id'].apply(lambda x: to_concept_id(regd[x], sub='[/ -\.\*";\[\]]+') )
income_group['name'] = income_group['income_group_id'].apply(lambda x: regd[x] )

income_group['is--income_group'] = 'TRUE'

In [226]:
income_group

,income_group,name,income_group_id,is--income_group
0,high_income,High income,i268,TRUE
1,lower_middle_income,Lower middle income,i269,TRUE
2,upper_middle_income,Upper middle income,i267,TRUE
3,low_income,Low income,i266,TRUE


In [237]:
income_group.to_csv('./output/ddf--entities--geo--income_group.csv', index=False)

In [227]:
landlocked = pd.DataFrame([], columns=['landlocked', 'name', 'landlocked_id', 'is--landlocked'])

landlocked['landlocked_id'] = gps.iloc[2]['groupings'].keys()
landlocked['landlocked'] = landlocked['landlocked_id'].apply(lambda x: to_concept_id(regd[x], sub='[/ -\.\*";\[\]]+') )
landlocked['name'] = landlocked['landlocked_id'].apply(lambda x: regd[x] )

landlocked['is--landlocked'] = 'TRUE'

In [228]:
landlocked

,landlocked,name,landlocked_id,is--landlocked
0,coastline,Coastline,i270,TRUE
1,landlocked,Landlocked,i271,TRUE


In [238]:
landlocked.to_csv('./output/ddf--entities--geo--landlocked.csv', index=False)

In [231]:
g77_and_oecd_countries = pd.DataFrame([], columns=['g77_and_oecd_countries', 'name', 'g77_and_oecd_countries_id', 'is--g77_and_oecd_countries'])

g77_and_oecd_countries['g77_and_oecd_countries_id'] = gps.iloc[3]['groupings'].keys()
g77_and_oecd_countries['g77_and_oecd_countries'] = g77_and_oecd_countries['g77_and_oecd_countries_id'].apply(lambda x: to_concept_id(regd[x], sub='[/ -\.\*";\[\]]+') )
g77_and_oecd_countries['name'] = g77_and_oecd_countries['g77_and_oecd_countries_id'].apply(lambda x: regd[x] )

g77_and_oecd_countries['is--g77_and_oecd_countries'] = 'TRUE'

In [232]:
g77_and_oecd_countries

,g77_and_oecd_countries,name,g77_and_oecd_countries_id,is--g77_and_oecd_countries
0,oecd,OECD,i273,TRUE
1,g77,G77,i272,TRUE
2,others,Others,i274,TRUE


In [239]:
g77_and_oecd_countries.to_csv('./output/ddf--entities--geo--g77_and_oecd_countries.csv', index=False)

In [242]:
geographic_regions_in_4_colors = pd.DataFrame([], columns=['geographic_regions_in_4_colors', 'name', 'geographic_regions_in_4_colors_id', 'is--geographic_regions_in_4_colors'])

geographic_regions_in_4_colors['geographic_regions_in_4_colors_id'] = gps.iloc[4]['groupings'].keys()
geographic_regions_in_4_colors['geographic_regions_in_4_colors'] = geographic_regions_in_4_colors['geographic_regions_in_4_colors_id'].apply(lambda x: to_concept_id(regd[x], sub='[/ -\.\*";\[\]]+', sep=''))
geographic_regions_in_4_colors['name'] = geographic_regions_in_4_colors['geographic_regions_in_4_colors_id'].apply(lambda x: regd[x] )

geographic_regions_in_4_colors['is--geographic_regions_in_4_colors'] = 'TRUE'

In [243]:
geographic_regions_in_4_colors

,geographic_regions_in_4_colors,name,geographic_regions_in_4_colors_id,is--geographic_regions_in_4_colors
0,africa,[Africa],i277,TRUE
1,europe,[Europe],i275,TRUE
2,america,[America],i278,TRUE
3,asia,[Asia],i276,TRUE


In [244]:
geographic_regions_in_4_colors.to_csv('./output/ddf--entities--geo--geographic_regions_in_4_colors.csv', index=False)

In [234]:
main_religion_2008 = pd.DataFrame([], columns=['main_religion_2008', 'name', 'main_religion_2008_id', 'is--main_religion_2008'])

main_religion_2008['main_religion_2008_id'] = gps.iloc[5]['groupings'].keys()
main_religion_2008['main_religion_2008'] = main_religion_2008['main_religion_2008_id'].apply(lambda x: to_concept_id(regd[x], sub='[/ -\.\*";\[\]]+') )
main_religion_2008['name'] = main_religion_2008['main_religion_2008_id'].apply(lambda x: regd[x] )

main_religion_2008['is--main_religion_2008'] = 'TRUE'

In [235]:
main_religion_2008

,main_religion_2008,name,main_religion_2008_id,is--main_religion_2008
0,christian,Christian,i279,TRUE
1,eastern_religions,Eastern religions,i281,TRUE
2,muslim,Muslim,i280,TRUE


In [245]:
main_religion_2008.to_csv('./output/ddf--entities--geo--main_religion_2008.csv', index=False)